
# Mobius Plateau – Auto‑Scan Notebook
Generated on demand. Drop into Colab and **Runtime → Run all**.

**Pipeline**

1. Load existing `plateaus_raw*.xlsx` if present; else seed minimal rows.
2. Define a high‑precision constant library (α, φ, π, e, γ, Feigenbaum δ/α).
3. Apply an exhaustive arithmetic operator library to every raw `rel_err`.
4. Detect ≥5‑digit threads against each constant’s digit string.
5. Persist full tables to Excel (`plateaus_raw_auto.xlsx`, `plateaus_ops_auto.xlsx`).

No external SymPy imports – everything uses `mpmath` (100 dps).


In [30]:

import pandas as pd, numpy as np, mpmath as mp, os, glob, re
mp.mp.dps = 100


In [31]:

# High‑precision constants
consts = {
    'alpha'      : mp.mpf('0.0072973525693'),
    'alpha_inv'  : mp.mpf('137.035999084'),
    'phi'        : (1 + mp.sqrt(5)) / 2,
    'phi_inv'    : 1 / ((1 + mp.sqrt(5)) / 2),
    'pi'         : mp.pi,
    'e'          : mp.e,
    'gamma'      : mp.euler,
    'feig_delta' : mp.mpf('4.66920160910299067185'),
    'feig_alpha' : mp.mpf('2.502907875095892822283902873218'),
}


In [32]:

# Seed or load raw plateaus
raw_files = glob.glob('/mnt/data/plateaus_raw*.xlsx')
if raw_files:
    df_raw = pd.read_excel(raw_files[-1])
else:
    seed_rows = [
        (64, 3.6, mp.mpf('0.003942'), 0.0, mp.mpf('0.4117525875906678269')),
        (64, 3.8, mp.mpf('0.003942'), 0.0, mp.mpf('0.4117525875906678269')),
        (64, 4.0, mp.mpf('0.003942'), 0.0, mp.mpf('0.4117525875906678269')),
        (64, 4.2, mp.mpf('0.003942'), 0.0, mp.mpf('0.4117525875906678269')),
        (128, 3.6, mp.mpf('0.003942'), 0.0, mp.mpf('0.4117525875906678271')),
        (128, 3.8, mp.mpf('0.003942'), 0.0, mp.mpf('0.4117525875906678271')),
        (128, 4.0, mp.mpf('0.003942'), 0.0, mp.mpf('0.4117525875906678271')),
        (128, 4.2, mp.mpf('0.003942'), 0.0, mp.mpf('0.4117525875906678271')),
    ]
    df_raw = pd.DataFrame(seed_rows, columns=['bits','k','phi_micro','lambda','rel_err'])


In [33]:

# Operator library
alpha = consts['alpha']; alpha_inv = consts['alpha_inv']
phi = consts['phi']; phi_inv = consts['phi_inv']
delta = consts['feig_delta']; feig_A = consts['feig_alpha']

ops = {
    '×9'      : lambda x: x*9,
    '÷9'      : lambda x: x/9,
    'sqrt'    : lambda x: mp.sqrt(x) if x >= 0 else mp.nan,
    'cbrt'    : lambda x: mp.nthroot(x, 3) if x >= 0 else mp.nan,
    'inv'     : lambda x: mp.nan if x==0 else 1/x,
    '1‑inv'   : lambda x: mp.nan if x==0 else 1 - 1/x,
    '×α'      : lambda x: x*alpha,
    '÷α'      : lambda x: x/alpha,
    '×α⁻¹'    : lambda x: x*alpha_inv,
    '÷α⁻¹'    : lambda x: x/alpha_inv,
    '×φ'      : lambda x: x*phi,
    '÷φ'      : lambda x: x/phi,
    '×δ'      : lambda x: x*delta,
    '÷δ'      : lambda x: x/delta,
    'pow_1/9' : lambda x: mp.power(x, 1/9),
    'pow_9'   : lambda x: mp.power(x, 9),
}


In [34]:

records = []
for _,row in df_raw.iterrows():
    base = mp.mpf(row['rel_err'])
    for op_name,fn in ops.items():
        try:
            val = fn(base)
        except Exception:
            val = mp.nan
        records.append({'bits':row['bits'],'k':row['k'],'base':base,'op':op_name,'val':val})
df_ops = pd.DataFrame(records)


In [35]:
# Digit thread detection (≥5‑digit substring)
def digits_only(x):
    # Use mpmath.nstr to format the mpf object
    return re.sub('[^0-9]', '', mp.nstr(x, n=100, strip_zeros=False))

targets = {name: digits_only(v) for name, v in consts.items()}

hits = []
for _,r in df_ops.iterrows():
    d = digits_only(r['val'])
    for name, ds in targets.items():
        # Ensure ds is not empty before checking for substring
        if ds and ds[:5] in d:
            hits.append({**r, 'constant':name, 'thread':ds[:5]})
df_hits = pd.DataFrame(hits)

In [36]:
# Save
raw_path = '/mnt/data/plateaus_raw_auto.xlsx'
ops_path = '/mnt/data/plateaus_ops_auto.xlsx'

# Create the directory if it doesn't exist
os.makedirs('/mnt/data', exist_ok=True)

df_raw.to_excel(raw_path, index=False)
df_ops.to_excel(ops_path, index=False)
print('Saved', raw_path, 'and', ops_path)
print('Thread hits found:', len(df_hits))

Saved /mnt/data/plateaus_raw_auto.xlsx and /mnt/data/plateaus_ops_auto.xlsx
Thread hits found: 0


In [37]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful